In [22]:
import torch
import numpy as np 
import os
import pandas as pd
import pickle

In [23]:
# loading meltome dataset in

data_filepath = "../datasets/meltome_full.csv"
df = pd.read_csv(data_filepath)
df.head()

,proteinID,meltTemp,temp&species,sequence
0,C0H3V2,56.693844,"[(56.6938442790629, 'Bacillus subtilis_168_lys...",MQVLAKENIKLNQTVSSKEEAIKLAGQTLIDNGYVTEDYISKMFER...
1,C0H3V8,43.373369,"[(43.3733692335437, 'Bacillus subtilis_168_lys...",MADKDFGLNDIVEMKKPHPCGANSWKIIRMGMDIRIKCEGCSHSVM...
2,C0H3Y1,40.693719,"[(40.69371854862, 'Bacillus subtilis_168_lysat...",MADYFLTVFDPSGNTLVNEQFEAEHEEAAKTHGEALLKEKELHSHT...
3,C0H405,48.069618,"[(48.0696175471916, 'Bacillus subtilis_168_lys...",MRPEEITEMIEQLRTGAINELRVQKEHFLDFRAVLSKQDDFKHFRG...
4,C0H437,39.507236,"[(39.5072356461209, 'Bacillus subtilis_168_lys...",MEEKYETNGYDTSIVYDYKEYPDVKYGRCDNCDYTLFKSSVKSGIF...


In [43]:
# loading in protein ID to melting temp dictionary

protID2MT = pickle.load(open("../datasets/protID2MT.p", "rb"))
dict_path = "../datasets/protID2MT.p"

In [39]:
# walking through unirep vec directory to find all available proteinIDs

URdir = "../datasets/UniRep_Vecs"

filenames = []

for root, dirs, files in os.walk(URdir):
   for name in files:
      filenames.append(name)

In [48]:
# creating dataloader
from torch.utils.data import Dataset, DataLoader
import pickle
import os

class MeltomeUnirepDataset(Dataset):

    def __init__(self, URdir, dictPath):
        '''
        Args:
           URdir (string): path to the unirep vector directory
           dictPath (string): path to the protein ID to melting temp dictionary
        '''
        self.unirepVecs = []

        for root, dirs, files in os.walk(URdir):
            for name in files:
                self.unirepVecs.append(name)

        self.protID2MT = pickle.load(open(dict_path, "rb"))
    def __len__(self):
        return len(self.unirepVecs)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        print(self.unirepVecs[idx])
        vec = np.load(os.path.join(URdir, self.unirepVecs[idx]))
        meltingTemp = self.protID2MT[self.unirepVecs[idx].replace(".npy", "")]
        output = {"vec": vec, "meltingTemp": meltingTemp}

        return output
        

In [49]:
# tester code
test_DS = MeltomeUnirepDataset(URdir=URdir, dictPath=dict_path)